In [2]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

## Настройки API

In [3]:
API_URL = "http://localhost:8000"
API_V1 = f"{API_URL}/api/v1"

# Эндпоинты
HEALTH_ENDPOINT = f"{API_V1}/health"
FORWARD_ENDPOINT = f"{API_V1}/forward"
BATCH_ENDPOINT = f"{API_V1}/forward/batch"

print(f"Health: {HEALTH_ENDPOINT}")
print(f"Forward: {FORWARD_ENDPOINT}")
print(f"Batch: {BATCH_ENDPOINT}")

Health: http://localhost:8000/api/v1/health
Forward: http://localhost:8000/api/v1/forward
Batch: http://localhost:8000/api/v1/forward/batch


## Загрузка данных

In [4]:
df = pd.read_pickle("../datasets/clean_ptbxl_with_ecg_n_diagnostic_superclass.pkl")
df.head(3)

,ecg_id,patient_id,age,sex,height,weight,device,recording_date,report,scp_codes,...,infarction_stadium1,infarction_stadium2,second_opinion,initial_autogenerated_report,validated_by_human,extra_beats,pacemaker,strat_fold,diagnostic_superclass,ecg_signals
1,2,13243.0,19.0,0,NaN,70.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",...,NaN,NaN,False,False,True,NaN,NaN,2,[NORM],"[[-0.015, 0.12, 0.135, -0.053, -0.075, 0.127, ..."
2,3,20372.0,37.0,1,NaN,69.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,False,False,True,NaN,NaN,5,[NORM],"[[-0.035, -0.07, -0.035, 0.053, 0.0, -0.052, 0..."
6,7,16193.0,54.0,0,NaN,83.0,CS-12 E,1984-11-28 13:32:22,"sinusrhythmus linkstyp t abnormal, wahrscheinl...","{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,False,False,True,NaN,NaN,7,[NORM],"[[-0.09, -0.02, 0.07, 0.055, -0.08, 0.025, 0.0..."


In [5]:
# функции для отправки запроса
def create_payload(row: pd.Series) -> dict:
    """
    Создает payload для запроса к API.
    
    Args:
        row: Строка DataFrame с данными пациента
        
    Returns:
        Словарь с данными для запроса
    """
    # обрабатываем heart_axis (заполняем NO_DATA если пропуск)
    heart_axis = row.get("heart_axis", "NO_DATA")
    if pd.isna(heart_axis):
        heart_axis = "NO_DATA"
    
    return {
        "age": float(row["age"]),
        "sex": int(row["sex"]),
        "height": float(row["height"]) if pd.notna(row["height"]) else 170.0,
        "weight": float(row["weight"]) if pd.notna(row["weight"]) else 70.0,
        "heart_axis": heart_axis,
        "ecg_signal": row["ecg_signals"].tolist()
    }


def send_prediction_request(row: pd.Series) -> dict:
    """
    Отправляет запрос к API для предсказания.
    
    Args:
        row: Строка DataFrame с данными пациента
        
    Returns:
        Ответ API
    """
    payload = create_payload(row)
    response = requests.post(
        FORWARD_ENDPOINT,
        json=payload,
        headers={"Content-Type": "application/json"}
    )
    return response

In [6]:
# берем первую запись для теста
test_row = df.iloc[0]

print(f"ECG ID: {test_row['ecg_id']}")
print(f"Возраст: {test_row['age']}")
print(f"Пол: {test_row['sex']}")
print(f"Рост: {test_row['height']}")
print(f"Вес: {test_row['weight']}")
print(f"Heart Axis: {test_row['heart_axis']}")
print(f"Диагноз: {test_row['diagnostic_superclass']}")
print(f"ECG shape: {test_row['ecg_signals'].shape}")

ECG ID: 2
Возраст: 19.0
Пол: 0
Рост: nan
Вес: 70.0
Heart Axis: nan
Диагноз: ['NORM']
ECG shape: (5000, 12)


In [7]:
# отправляем запрос на сервер
response = send_prediction_request(test_row)

print(f"Статус код: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"\nРезультат предсказания:")
    print(f"  Класс: {result['class_name']}")
    print(f"  Индекс: {result['class_index']}")
    print(f"  Описание: {result['class_description']}")
else:
    print(f"Ошибка: {response.json()}")

Статус код: 200

Результат предсказания:
  Класс: N
  Индекс: 0
  Описание: Normal ECG


## Тестирование на нескольких записях

In [8]:
# тестируем на первых N записях
N_SAMPLES = 100

results = []

for idx in tqdm(range(N_SAMPLES), desc="Отправка запросов"):
    row = df.iloc[idx]
    response = send_prediction_request(row)
    
    if response.status_code == 200:
        result = response.json()
        results.append({
            "ecg_id": row["ecg_id"],
            "actual_diagnosis": str(row["diagnostic_superclass"]),
            "predicted_class": result["class_name"],
            "predicted_index": result["class_index"],
            "predicted_description": result["class_description"],
            "status": "success"
        })
    else:
        results.append({
            "ecg_id": row["ecg_id"],
            "actual_diagnosis": str(row["diagnostic_superclass"]),
            "predicted_class": None,
            "predicted_index": None,
            "predicted_description": None,
            "status": f"error: {response.status_code}"
        })

results_df = pd.DataFrame(results)
print(f"\nОбработано записей: {len(results_df)}")
results_df

Отправка запросов: 100%|██████████| 100/100 [03:35<00:00,  2.16s/it]


Обработано записей: 100


,ecg_id,actual_diagnosis,predicted_class,predicted_index,predicted_description,status
0,2,['NORM'],N,0,Normal ECG,success
1,3,['NORM'],N,0,Normal ECG,success
2,7,['NORM'],CS,5,Conduction Disturbance + ST/T Change,success
3,10,['NORM'],N,0,Normal ECG,success
4,12,['NORM'],N,0,Normal ECG,success
...,...,...,...,...,...,...
95,171,['STTC'],N,0,Normal ECG,success
96,172,['CD'],MC,4,Myocardial Infarction + Conduction Disturbance,success
97,176,['NORM'],S,1,ST/T Change,success
98,177,['MI'],MC,4,Myocardial Infarction + Conduction Disturbance,success


In [9]:
# анализ результатов
print("Статистика:")
print(f"Всего запросов: {len(results_df)}")
print(f"Успешных: {(results_df['status'] == 'success').sum()}")
print(f"С ошибками: {(results_df['status'] != 'success').sum()}")

print(f"\nРаспределение предсказанных классов:")
print(results_df['predicted_class'].value_counts())

Статистика:
Всего запросов: 100
Успешных: 100
С ошибками: 0

Распределение предсказанных классов:
predicted_class
N      65
MC      7
CS      6
M       5
H       4
S       4
C       3
MS      2
MHS     2
MH      1
MHC     1
Name: count, dtype: int64


In [10]:
# cоздаем batch запрос (отправляем несколько записей за раз)
batch_size = 100
batch_samples = [create_payload(df.iloc[i]) for i in range(batch_size)]

batch_payload = {"samples": batch_samples}

print(f"Отправляем batch запрос с {len(batch_samples)} записями...")

response = requests.post(
    BATCH_ENDPOINT,
    json=batch_payload,
    headers={"Content-Type": "application/json"}
)

print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    batch_result = response.json()
    print(f"\nПолучено {len(batch_result['predictions'])} предсказаний:")
    for i, pred in enumerate(batch_result['predictions']):
        print(f"  {i+1}. {pred['class_name']} - {pred['class_description']}")
else:
    print(f"Ошибка: {response.json()}")

Отправляем batch запрос с 100 записями...
Статус код: 200

Получено 100 предсказаний:
  1. N - Normal ECG
  2. N - Normal ECG
  3. CS - Conduction Disturbance + ST/T Change
  4. N - Normal ECG
  5. N - Normal ECG
  6. N - Normal ECG
  7. N - Normal ECG
  8. CS - Conduction Disturbance + ST/T Change
  9. N - Normal ECG
  10. N - Normal ECG
  11. M - Myocardial Infarction
  12. N - Normal ECG
  13. N - Normal ECG
  14. CS - Conduction Disturbance + ST/T Change
  15. N - Normal ECG
  16. N - Normal ECG
  17. N - Normal ECG
  18. N - Normal ECG
  19. N - Normal ECG
  20. CS - Conduction Disturbance + ST/T Change
  21. M - Myocardial Infarction
  22. MC - Myocardial Infarction + Conduction Disturbance
  23. N - Normal ECG
  24. N - Normal ECG
  25. N - Normal ECG
  26. N - Normal ECG
  27. H - Hypertrophy
  28. N - Normal ECG
  29. MS - Myocardial Infarction + ST/T Change
  30. N - Normal ECG
  31. N - Normal ECG
  32. N - Normal ECG
  33. N - Normal ECG
  34. N - Normal ECG
  35. H - Hyper

## Тест | Обработка Ошибок

In [11]:
# неверный heart_axis
print("Тест 1: Неверное значение heart_axis")

bad_payload = {
    "age": 50,
    "sex": 1,
    "height": 175,
    "weight": 80,
    "heart_axis": "INVALID_VALUE",
    "ecg_signal": [[0.1] * 12 for _ in range(100)]
}

response = requests.post(FORWARD_ENDPOINT, json=bad_payload)
print(f"  Статус код: {response.status_code}")
print(f"  Ожидаемый: 400 (Bad Request)")

Тест 1: Неверное значение heart_axis
  Статус код: 400
  Ожидаемый: 400 (Bad Request)


In [12]:
# тест с отсутствующими полями
print("Тест 2: Отсутствуют обязательные поля")

incomplete_payload = {
    "age": 50,
    "sex": 1
}

response = requests.post(FORWARD_ENDPOINT, json=incomplete_payload)
print(f"  Статус код: {response.status_code}")
print(f"  Ожидаемый: 400 (Bad Request)")

Тест 2: Отсутствуют обязательные поля
  Статус код: 400
  Ожидаемый: 400 (Bad Request)


In [13]:
# тест с неверным количеством отведений
print("Тест 3: Неверное количество отведений (не 12)")

wrong_leads_payload = {
    "age": 50,
    "sex": 1,
    "height": 175,
    "weight": 80,
    "heart_axis": "MID",
    "ecg_signal": [[0.1] * 5 for _ in range(100)]  # 5 отведений вместо 12
}

response = requests.post(FORWARD_ENDPOINT, json=wrong_leads_payload)
print(f"  Статус код: {response.status_code}")
print(f"  Ожидаемый: 400 (Bad Request)")

Тест 3: Неверное количество отведений (не 12)
  Статус код: 400
  Ожидаемый: 400 (Bad Request)


## Замер времени отклика сервиса

In [14]:
import time

# замеряем время для одного запроса
test_row = df.iloc[0]
payload = create_payload(test_row)

times = []
n_requests = 10

print(f"Замер времени отклика ({n_requests} запросов)...")

for _ in tqdm(range(n_requests)):
    start = time.time()
    response = requests.post(FORWARD_ENDPOINT, json=payload)
    end = time.time()
    times.append(end - start)

print(f"\nРезультаты:")
print(f"  Среднее время: {np.mean(times)*1000:.2f} мс")
print(f"  Мин: {np.min(times)*1000:.2f} мс")
print(f"  Макс: {np.max(times)*1000:.2f} мс")
print(f"  Std: {np.std(times)*1000:.2f} мс")

Замер времени отклика (10 запросов)...


100%|██████████| 10/10 [00:21<00:00,  2.14s/it]


Результаты:
  Среднее время: 2143.84 мс
  Мин: 2117.21 мс
  Макс: 2179.07 мс
  Std: 18.68 мс
